# Track military vehicles in video inputs

In this notebook, we'll use the model that we trained with our custom dataset to track military vehicles in videos. We'll use the [YOLO](https://github.com/ultralytics/ultralytics) library to provide state-of-the-art multi-object tracking algorithms.

### Setup

To start, check GPU support.

In [9]:
import torch

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.3.1+cu121 (NVIDIA RTX A6000)


### Run inference on the video

We run our detection model on the sample video, saving the output to disk.

In [10]:
import pickle

# Load the best_model variable from the file
with open('best_model.pkl', 'rb') as f:
    best_model = pickle.load(f)

print(best_model)

runs/detect/train/weights/best.pt


In [11]:
from pathlib import Path
from adomvi.yolo.yolo import track

video_path = Path() / "../resources/test/apc1.mp4"
results_track = track(best_model, source=video_path, tracker="bytetrack.yaml")



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/511) /home/ukemkata/workspace/adomvi2/notebooks/../resources/test/apc1.mp4: 384x640 1 APC, 22.5ms
video 1/1 (frame 2/511) /home/ukemkata/workspace/adomvi2/notebooks/../resources/test/apc1.mp4: 384x640 1 APC, 8.6ms
video 1/1 (frame 3/511) /home/ukemkata/workspace/adomvi2/notebooks/../resources/test/apc1.mp4: 384x640 1 APC, 8.7ms
video 1/1 (frame 4/511) /home/ukemkata/workspace/adomvi2/notebooks/../resources/test/apc1.mp4: 384x640 1 AP

In [12]:
# Load the path of the track model results

results_track_dir = Path(results_track[0].save_dir)

### Display the result video

Compress the video file to a format supported by HTML (mpeg4 (native) -> h264 (libx264))

In [13]:
from adomvi.utils import convert_video_avi_to_mp4

input_video_path = Path() / results_track_dir / "apc1.avi"
output_video_path = Path() / results_track_dir / "apc1.mp4"
convert_video_avi_to_mp4(input_video_path, output_video_path)

Moviepy - Building video runs/detect/track13/apc1.mp4.
Moviepy - Writing video runs/detect/track13/apc1.mp4



Moviepy - Done !
Moviepy - video ready runs/detect/track13/apc1.mp4


Display it with HTML

In [14]:
import IPython.display as display
from base64 import b64encode

video_url = (f"data:video/mp4;base64,{b64encode(output_video_path.read_bytes()).decode()}")
display.HTML(
    f"""
    <video controls width="640">
        <source src="{video_url}" type="video/mp4">
    </video>
    """
)